In [22]:
from dtsp.dataset import walk_forward_split, Seq2SeqDataSet, SimpleSeq2SeqDataSet, SimpleWaveNetDataSet
from dtsp.models import Seq2Seq, SimpleSeq2Seq, SimpleWaveNet
from torch.utils.data import DataLoader
from torch.utils.data import Subset, DataLoader
import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from pathlib import Path

%matplotlib inline

In [3]:
df = pd.read_csv('./data.csv', index_col='date_time', parse_dates=['date_time'])
series = df.values
month = df.index.month.values

In [6]:
df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
date_time,,,,,,,,,,,,,,,,,,,,
1998-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0,0
1998-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0,0
1998-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0,0
1998-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0,0
1998-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0,0


In [7]:
idxes = list(range(len(series)))

In [18]:
n_test = 12
n_val = 12
enc_lens = 72
dec_lens = 12
batch_size = 8

In [19]:
dataset = Seq2SeqDataSet(series, enc_lens, dec_lens, categorical_var=month)
idxes = list(range(len(dataset)))
train_idxes, _idxes = walk_forward_split(idxes, enc_lens, dec_lens, test_size=n_test + n_val)
valid_idxes, test_idxes = walk_forward_split(_idxes, enc_lens, dec_lens, test_size=n_test)

trn_set = Subset(dataset, train_idxes)
val_set = Subset(dataset, valid_idxes)
test_set = Subset(dataset, test_idxes)
trn_ld = DataLoader(trn_set, batch_size=batch_size, shuffle=True, drop_last=False)
val_ld = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=False)
test_ld = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [25]:
hp = {
        'path': Path('.').resolve() / 'logs',
        'target_size': 20,
        'rnn_type': 'LSTM',
        'dropout': 0.1,
        'hidden_size': 128,
        'teacher_forcing_rate': 0.5,
        'n_head': 2,
        'use_attn': False,
        'trans_hidden_size': 4,
        'trans_continuous_var': None,
        'trans_category_var': [(13, 2)],
        'trans_bidirectional': True,
        'trans_rnn_type': 'LSTM',
        'use_move_scale': True,
    }

compile_params = {
    'loss_fn': 'MSELoss',
    'optimizer': 'Adam',
    'lr': 0.001,
    'lr_scheduler': 'CosineAnnealingWarmRestarts',
    'lr_scheduler_kw': {'T_0': 5, 'T_mult': 10},
    'metric': 'RMSE',
}

In [26]:
model = Seq2Seq(hp)
model.compile(**compile_params)
model.fit(50, trn_ld, val_ld, early_stopping=10, save_every_n_epochs=None, save_best_model=True)

batch 2 / 17, loss 0.944:  12%|█▏        | 2/17 [00:00<00:01, 13.13it/s]

epoch 1 / 50: train loss 1.034 val loss 1.137 RMSE 177.692


batch 3 / 17, loss 0.812:  12%|█▏        | 2/17 [00:00<00:00, 15.41it/s]

epoch 2 / 50: train loss 0.857 val loss 0.981 RMSE 167.018


batch 2 / 17, loss 0.656:  12%|█▏        | 2/17 [00:00<00:01, 14.69it/s]

...

epoch 48 / 50: train loss 0.246 val loss 0.478 RMSE 106.177


batch 3 / 17, loss 0.192:  12%|█▏        | 2/17 [00:00<00:00, 16.75it/s]

epoch 49 / 50: train loss 0.244 val loss 0.475 RMSE 105.842


batch 17 / 17, loss 0.241: 100%|██████████| 17/17 [00:01<00:00, 15.58it/s]


epoch 50 / 50: train loss 0.241 val loss 0.475 RMSE 105.761
best model save in /home/zhouzr/project/Deep-Time-Series-Prediction/notebooks/logs/Seq2Seq_epoch_49_0.475.pkl


In [28]:
model.reload(model.best_model_path())
print(' - ' * 20)
print(f'train loss: {model.eval_cycle(trn_ld)[0]:.3f}, '
      f'valid loss: {model.eval_cycle(val_ld)[0]:.3f}, '
      f'test loss :{model.eval_cycle(test_ld)[0]:.3f}, '
      f'test RMSE :{model.eval_cycle(test_ld)[1]:.3f}, ')

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
train loss: 0.247, valid loss: 0.475, test loss :0.555, test RMSE :116.132, 


In [43]:
len(series)

239

In [59]:
pred_idx = 200
enc_x = torch.tensor(series[pred_idx - enc_lens: pred_idx]).unsqueeze(0).float()
cat_x = torch.tensor(month[pred_idx-enc_lens: pred_idx + dec_lens]).unsqueeze(0).unsqueeze(2).long()

model.predict(enc_x, dec_lens, category_x=cat_x, use_move_scale=True)

AttributeError: 'list' object has no attribute 'shape'

In [61]:
enc_x.shape

torch.Size([1, 72, 20])